In [ ]:
import numpy as np
from numpy import genfromtxt
import glob
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import sys
pd.set_option('display.max_rows', 1000)
import random
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from scipy import stats
import matplotlib.pyplot as plt
from keras.models import Model
import math

In [ ]:
data_folder = "data/RawData/"
acc_path = data_folder + "acc_*.txt"
gyro_path = data_folder + "gyro_*.txt"
label_path = data_folder + "labels.txt"

df_labels = pd.read_csv(label_path, delimiter=r"\s+")

df_main = pd.DataFrame(columns=['acc0','acc1','acc2','gyro0','gyro1','gyro2','exp_id','label_id','subject_id'])

acc_files = glob.glob(acc_path)
for acc_file in acc_files:
    
    exp_id = acc_file[20:22]
    subject_id = acc_file[27:29]
    df_acc = pd.read_csv(acc_file, header=None, delimiter=r"\s+", prefix="acc")
    gyro_file = data_folder + "gyro" + acc_file[len(data_folder)+3:]
    df_gyro = pd.read_csv(gyro_file, header=None, delimiter=r"\s+", prefix="gyro")
    
    if len(df_acc) != len(df_gyro):
        print ("Error: lenght of acc and gyro files are different for exp " + exp_id + ": " 
               + str(len(df_acc)) + " " + str(len(df_gyro)))
    
    df_concat = pd.concat([df_acc, df_gyro], axis = 1)
    df_concat["exp_id"] = int(exp_id)
    df_concat["label_id"] = 0
    df_concat["subject_id"] = int(subject_id)

    df_exp_labels = df_labels.loc[df_labels['exp_id'] == int(exp_id)]

    for index, row in df_exp_labels.iterrows():
        start_point = row['start_point']
        end_point   = row['end_point']
        label_id    = row['act_id']
        df_concat.loc[start_point:end_point , 'label_id'] = label_id

    df_main = df_main.append(df_concat, ignore_index = True)

In [ ]:
window_size = 128
window_move_size = window_size // 2

In [ ]:
def segment(df_OVA):

    segmeneted_activities = [ 'N/A',[],[],[],[],[],[] ]

    for i in range(1 , 62):
    
        df_exp = df_OVA.loc[df_OVA['exp_id'] == i]
        df_exp = df_exp.reset_index(drop=True)
    
        index = 0
        continu = True
        while (continu):
        
            if index >= len(df_exp):
                break
        
            df_window = df_exp.loc[index:index+window_size-1]
            if len(df_window) < window_size:
                break
        
            window_label_ids = set(df_window['label_id'].values)
            if (len(window_label_ids) > 1) or \
               (len(window_label_ids)==1 and next(iter(window_label_ids))==0) or \
               (len(window_label_ids)==1 and next(iter(window_label_ids))>6):
                index = index + window_move_size
                continue
    
            na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
            window_label_id = next(iter(window_label_ids))
        
            segmeneted_activities[window_label_id].append(na_window)
            index = index + window_move_size
            
    for i in range(1,7):
        segmeneted_activities[i] = np.array(segmeneted_activities[i])

    for i in range(1,7):
        print ("Act_id:" , i , " " , segmeneted_activities[i].shape)
        
    return segmeneted_activities

In [ ]:
def get_binary_X_and_Y_oversampled(act_id, segmeneted_activities):
    
    X , Y = get_binary_X_and_Y (act_id, segmeneted_activities)

    total_datapoints = X.shape[0]
    plus_datapoints = segmeneted_activities[act_id].shape[0]
    alpha = (total_datapoints - plus_datapoints) / plus_datapoints
    alpha = round(alpha, 1)
    fraction = int(str(alpha)[-1])
    alpha = int(alpha)
    plus_class_counter = -1
    
    New_X = []
    New_Y = []

    for i in range(total_datapoints):
        
        if (Y[i]==1):
            plus_class_counter += 1
            for j in range(alpha):
                New_X.append(X[i])
                New_Y.append(Y[i])

            if (plus_class_counter % 10) < fraction:
                New_X.append(X[i])
                New_Y.append(Y[i])
                
        else:
            New_X.append(X[i])
            New_Y.append(Y[i])

    New_X = np.array(New_X)
    New_Y = np.array(New_Y)
    
    return New_X , New_Y

In [ ]:
def get_binary_X_and_Y (act_id, segmeneted_activities):

    X = np.concatenate((segmeneted_activities[1], segmeneted_activities[2], segmeneted_activities[3]))
    X = np.concatenate((           X            , segmeneted_activities[4], segmeneted_activities[5]))
    X = np.concatenate((           X            , segmeneted_activities[6]))

    ys = [ 'N/A',None,None,None,None,None,None ]
    for i in range(1,7):
        size = segmeneted_activities[i].shape[0]
        if act_id == i:
            y = np.full (size , 1)
        else:
            y = np.full (size , 0)
        ys[i] = y

    Y = np.concatenate((ys[1], ys[2], ys[3]))
    Y = np.concatenate((  Y  , ys[4], ys[5]))
    Y = np.concatenate((  Y  , ys[6]))
    
    return (X , Y)

In [ ]:
def create_model (data_input_shape):

    LSTM_output_dim = 100
    
    model = Sequential()
    
    #model.add(LSTM(LSTM_output_dim, return_sequences=True, name='LSTM1', input_shape=data_input_shape))    
    #model.add(LSTM(LSTM_output_dim, return_sequences=True, name='LSTM2'))
    #model.add(LSTM(LSTM_output_dim, return_sequences=False, name='LSTM3'))
    
    model.add(LSTM(LSTM_output_dim, return_sequences=False, name='LSTM1', input_shape=data_input_shape))
    
    model.add(Dense(15, activation='sigmoid',name='Hidden_Layer'))
    
    #model.add(Dense(3, activation='softmax', name='Softmax_layer'))
    model.add(Dense(1, activation='sigmoid',name='Dense_out'))
    
    #model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

    return model

In [ ]:
def train_binary_classifier(act_id, segmeneted_activities):
    #(X , Y) = get_binary_X_and_Y (act_id, segmeneted_activities)
    (X , Y) = get_binary_X_and_Y_oversampled (act_id , segmeneted_activities)
    
    X , Y = shuffle(X , Y)

    X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25)
    X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.333)

    data_input_shape = X.shape[1:3]
    model = create_model(data_input_shape)
    #print (model.summary()  , "\n")

    patience = 20
    callbacks=[EarlyStopping(monitor='val_loss',patience=patience,verbose=1)]
    history = model.fit(X_train,y_train,
                        validation_data=[X_valid,y_valid],
                        epochs=50, 
                        batch_size=100,
                        verbose=2
                        #,callbacks=callbacks
                       )

    score = model.evaluate(X_test, y_test, verbose=0)
    print('\nTest loss:', score[0])
    print('Test accuracy:', score[1])
    print()
    
    return model

In [ ]:
def train(df_OVA):
    segmeneted_activities = segment(df_OVA)

    #training binary classifier for activity 1 (WALKING)
    print ()
    print ("Training activity 1 ...")
    act_id = 1
    model_1 = train_binary_classifier(act_id, segmeneted_activities)

    #training binary classifier for activity 2 (WALKING_UPSTAIRS)
    print ()
    print ("Training activity 2 ...")
    act_id = 2
    model_2 = train_binary_classifier(act_id, segmeneted_activities)

    #training binary classifier for activity 3 (WALKING_DOWNSTAIRS)
    print ()
    print ("Training activity 3 ...")
    act_id = 3
    model_3 = train_binary_classifier(act_id, segmeneted_activities)

    #training binary classifier for activity 4 (SITTING)
    print ()
    print ("Training activity 4 ...")
    act_id = 4
    model_4 = train_binary_classifier(act_id, segmeneted_activities)

    #training binary classifier for activity 5 (STANDING)
    print ()
    print ("Training activity 5 ...")
    act_id = 5
    model_5 = train_binary_classifier(act_id, segmeneted_activities)

    #training binary classifier for activity 6 (LAYING)
    print ()
    print ("Training activity 6 ...")
    act_id = 6
    model_6 = train_binary_classifier(act_id, segmeneted_activities)

    models = []
    models.append(model_1)
    models.append(model_2)
    models.append(model_3)
    models.append(model_4)
    models.append(model_5)
    models.append(model_6)

    return models

In [ ]:
'''
model_1.save('model_1.h5')
model_2.save('model_2.h5')
model_3.save('model_3.h5')
model_4.save('model_4.h5')
model_5.save('model_5.h5')
model_6.save('model_6.h5')
'''

In [ ]:
def predict(X_predict, models):

    X_predict = np.reshape(X_predict , (1,X_predict.shape[0],X_predict.shape[1]))

    scores = [0, 0, 0, 0, 0, 0]
    for i in range(6):
        pred_probs = models[i].predict_proba(X_predict)
        scores[i] = pred_probs[0][0]

    threshold = 0.5
    if max(scores) < threshold:
        label_id = 0
    else:
        label_id = scores.index(max(scores)) + 1
    
    return label_id

In [ ]:
def test(df_continuous , models):

    # =================
    #df_continuous2 = df_continuous.loc[df_continuous['exp_id'] == 47]
    df_continuous2 = df_continuous
    # =================

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0
    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            print ("Window Label: 0")
            index = index + window_move_size
            continue
        
        na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
        pred_label_id = predict(na_window, models)

        print ("Window Label: " + str(window_label_id) + "  Predicted Label: " + str(pred_label_id))
        
        total_preds += 1
        
        if (pred_label_id == window_label_id) or \
           (window_label_id>6 and pred_label_id==0) or \
           (window_label_id==7 and (pred_label_id==5 or pred_label_id==4)) or \
           (window_label_id==8 and (pred_label_id==4 or pred_label_id==5)) or \
           (window_label_id==9 and (pred_label_id==4 or pred_label_id==6)) or \
           (window_label_id==10 and (pred_label_id==6 or pred_label_id==4)) or \
           (window_label_id==11 and (pred_label_id==5 or pred_label_id==6)) or \
           (window_label_id==12 and (pred_label_id==6 or pred_label_id==5)):
        
            correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

#### Cao et al Smoothing

In [ ]:
def get_u(df_OVA):

    run_lengths = []
    current_label = 0
    run_length = 1

    for i in range(1 , len(df_OVA)):
    
        label = df_OVA.loc[i , 'label_id']
    
        if current_label == label:
            run_length += 1
            continue
        else:
            if current_label != 0:
                run_lengths.append(run_length)
            run_length = 1
            current_label = label
            continue
        
    run_lengths = np.array(run_lengths)
    run_lengths_sorted = np.sort(run_lengths)
    p = 1. * np.arange(len(run_lengths)) / (len(run_lengths) - 1)

    #import matplotlib.pyplot as plt
    #plt.figure()
    #plt.plot(run_lengths_sorted, p)
    #plt.title('cumulative distribution function (CDF) of all run lengths')
    #plt.ylabel('p')
    #plt.xlabel('run length')
    #plt.show()
    
    thr = 0.03
    for i in range(p.shape[0]):
        if p[i] >= thr:
            break

    idx = i-1
    u = run_lengths_sorted[idx]
    #u = round((u - window_size) / (2.0 * window_move_size))
    return u

In [ ]:
def test_with_cao_smoothing_3 (df_continuous , models):

    df_continuous2 = df_continuous

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0
    l = []
    w = []
    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            l.append(0)
            w.append(0)
        
        else:    
            na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
            pred_label_id = predict(na_window, models)
            l.append(pred_label_id)
            w.append(window_label_id)
    
        i = len(l)
        if i > 2:
        
            if w[i-2]==0:
                print ("Window Label: 0")
                index = index + window_move_size
                continue
            
            elif w[i-1]==0:
                smoothed_pred_label_id = l[i-2]
            
            elif w[i-3]==0:
                smoothed_pred_label_id = l[i-2]
            
            else:
                pred_seg = [ l[i-3] , l[i-2] , l[i-1] ]
                mode = stats.mode(pred_seg)
                if mode[1][0] > 1:
                    smoothed_pred_label_id = mode[0][0]
                else:
                    smoothed_pred_label_id = l[i-2]
            
            print ("Window Label: " + str(w[i-2]) + "  Smoothed Predicted Label: " + str(smoothed_pred_label_id))
      
            total_preds += 1
        
            if (smoothed_pred_label_id == w[i-2]) or \
               (w[i-2]>6 and smoothed_pred_label_id==0) or \
               (w[i-2]==7 and (smoothed_pred_label_id==5 or smoothed_pred_label_id==4)) or \
               (w[i-2]==8 and (smoothed_pred_label_id==4 or smoothed_pred_label_id==5)) or \
               (w[i-2]==9 and (smoothed_pred_label_id==4 or smoothed_pred_label_id==6)) or \
               (w[i-2]==10 and (smoothed_pred_label_id==6 or smoothed_pred_label_id==4)) or \
               (w[i-2]==11 and (smoothed_pred_label_id==5 or smoothed_pred_label_id==6)) or \
               (w[i-2]==12 and (smoothed_pred_label_id==6 or smoothed_pred_label_id==5)):
        
                correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

#### Krishnan et al Smoothing

delta_t = 3

In [ ]:
def g(x):
    
    result = (1. / (math.sqrt(2*math.pi))) * np.exp(-0.5 * x * x)
    return result

In [ ]:
def get_window_feature_vector(X_predict , intermediate_layer_model):
    
    intermediate_output = intermediate_layer_model.predict(X_predict)
    intermediate_output = np.reshape(intermediate_output, (intermediate_output.shape[1]))
    
    return intermediate_output

In [ ]:
def euc_dist (f1 , f2):
    
    dist = np.linalg.norm(f2-f1)
    return dist

In [ ]:
def h(f1 , f2):

    alpha = 1
    result = np.exp(-alpha * euc_dist(f1 , f2))
    return result

In [ ]:
def get_predict_vector(na_window, models, intermediate_layer_models):
    
    num_of_classes = 6
    X_predict = na_window
    X_predict = np.reshape(X_predict , (1,X_predict.shape[0],X_predict.shape[1]))

    scores = [0, 0, 0, 0, 0, 0]
    f = [None, None, None, None, None, None]
    for i in range(num_of_classes):
        pred_probs = models[i].predict_proba(X_predict)
        scores[i] = pred_probs[0][0]

        f[i] = get_window_feature_vector(X_predict , intermediate_layer_models[i])
    
    return scores , f

In [ ]:
def test_with_Krishnan_smoothing_delta3(df_continuous , models):

    intermediate_layer_model_1 = Model(inputs=models[0].input, outputs=models[0].get_layer("LSTM1").output)
    intermediate_layer_model_2 = Model(inputs=models[1].input, outputs=models[1].get_layer("LSTM1").output)
    intermediate_layer_model_3 = Model(inputs=models[2].input, outputs=models[2].get_layer("LSTM1").output)
    intermediate_layer_model_4 = Model(inputs=models[3].input, outputs=models[3].get_layer("LSTM1").output)
    intermediate_layer_model_5 = Model(inputs=models[4].input, outputs=models[4].get_layer("LSTM1").output)
    intermediate_layer_model_6 = Model(inputs=models[5].input, outputs=models[5].get_layer("LSTM1").output)

    intermediate_layer_models = []
    intermediate_layer_models.append(intermediate_layer_model_1)
    intermediate_layer_models.append(intermediate_layer_model_2)
    intermediate_layer_models.append(intermediate_layer_model_3)
    intermediate_layer_models.append(intermediate_layer_model_4)
    intermediate_layer_models.append(intermediate_layer_model_5)
    intermediate_layer_models.append(intermediate_layer_model_6)

    df_continuous2 = df_continuous

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0

    num_of_classes = 6
    P = [[] , [] , [] , [] , [] , []]
    fs = [[] , [] , [] , [] , [] , []]
    delta_t = 3

    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            P = [[] , [] , [] , [] , [] , []]
            fs = [[] , [] , [] , [] , [] , []]
            print ("Window Label: 0")
            index = index + window_move_size
            continue
        
        na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
        p , f = get_predict_vector(na_window, models, intermediate_layer_models)
    
        t = len(P[0])
    
        if t < (delta_t):
            for i in range(num_of_classes):
                P[i].append(p[i])
                fs[i].append(f[i])
            
        else:
            for i in range(num_of_classes):
            
                fs[i].append(f[i])
            
                s1 = 0
                for j in range(1 , delta_t+1):
                    s1 += g(j) * h(fs[i][t-j] , fs[i][t]) * P[i][t-j]
            
                s2 = 0
                for j in range(1 , delta_t+1):
                    s2 += g(j) * h(fs[i][t-j] , fs[i][t])
            
                P[i].append( (p[i] + s1) / (s2 + 1) )
    
    
        current_P = []
        for i in range(num_of_classes):
            current_P.append ( P[i][t] )
    
        threshold = 0.5
        if max(current_P) < threshold:
            pred_label_id = 0
        else:
            pred_label_id = current_P.index(max(current_P)) + 1
    
        print ("Window Label: " + str(window_label_id) + "  Smoothed Predicted Label: " + str(pred_label_id))
    
        total_preds += 1
        
        if (pred_label_id == window_label_id) or \
           (window_label_id>6 and pred_label_id==0) or \
           (window_label_id==7 and (pred_label_id==5 or pred_label_id==4)) or \
           (window_label_id==8 and (pred_label_id==4 or pred_label_id==5)) or \
           (window_label_id==9 and (pred_label_id==4 or pred_label_id==6)) or \
           (window_label_id==10 and (pred_label_id==6 or pred_label_id==4)) or \
           (window_label_id==11 and (pred_label_id==5 or pred_label_id==6)) or \
           (window_label_id==12 and (pred_label_id==6 or pred_label_id==5)):
        
            correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

##### Simple Voting Scheme

N = 3

In [ ]:
def test_with_simple_voting_smoothing(df_continuous , models):

    df_continuous2 = df_continuous

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0

    num_of_classes = 6
    N = 3
    l = []

    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            l = []
            print ("Window Label: 0")
            index = index + window_move_size
            continue
        
        na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
        pred_label_id = predict(na_window, models)
        l.append(pred_label_id)

        t = len(l)
    
        if t < N:
            smoothed_pred_label_id = pred_label_id
            
        else:
            voting_list = []
            for i in range(N):
                voting_list.append( l[t-i-1] )
        
            mode = stats.mode(voting_list)
        
            if mode[1][0] > 1:
                smoothed_pred_label_id = mode[0][0]
            else:
                smoothed_pred_label_id = pred_label_id

    
        print ("Window Label: " + str(window_label_id) + "  Smoothed Predicted Label: " + str(smoothed_pred_label_id))
    
        total_preds += 1
        
        if (smoothed_pred_label_id == window_label_id) or \
           (window_label_id>6 and smoothed_pred_label_id==0) or \
           (window_label_id==7 and (smoothed_pred_label_id==5 or smoothed_pred_label_id==4)) or \
           (window_label_id==8 and (smoothed_pred_label_id==4 or smoothed_pred_label_id==5)) or \
           (window_label_id==9 and (smoothed_pred_label_id==4 or smoothed_pred_label_id==6)) or \
           (window_label_id==10 and (smoothed_pred_label_id==6 or smoothed_pred_label_id==4)) or \
           (window_label_id==11 and (smoothed_pred_label_id==5 or smoothed_pred_label_id==6)) or \
           (window_label_id==12 and (smoothed_pred_label_id==6 or smoothed_pred_label_id==5)):
        
            correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

##### Exponentially Weighted Voting

alpha = 0.05, 0.25

In [ ]:
def test_with_EWV_smoothing (df_continuous , models, alpha):

    df_continuous2 = df_continuous

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0

    num_of_classes = 7
    w = [ [] , [] , [] , [] , [] , [] , []]

    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            w = [ [] , [] , [] , [] , [] , [] , []]
            print ("Window Label: 0")
            index = index + window_move_size
            continue
        
        na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
        pred_label_id = predict(na_window, models)

        t = len(w[0])
    
        if t == 0:
            smoothed_pred_label_id = pred_label_id
            for i in range(num_of_classes):
                w[i].append(0)
            
        else:
            current_ws = [0,0,0,0,0,0,0]
            for i in range(num_of_classes):
            
                if i == pred_label_id:
                    c = 1
                else:
                    c = 0
                weight = w[i][t-1] + alpha * ( c - w[i][t-1])
                w[i].append(weight)
                current_ws[i] = weight
        
            smoothed_pred_label_id = current_ws.index(max(current_ws))

    
        print ("Window Label: " + str(window_label_id) + "  Smoothed Predicted Label: " + str(smoothed_pred_label_id))
    
        total_preds += 1
        
        if (smoothed_pred_label_id == window_label_id) or \
           (window_label_id>6 and smoothed_pred_label_id==0) or \
           (window_label_id==7 and (smoothed_pred_label_id==5 or smoothed_pred_label_id==4)) or \
           (window_label_id==8 and (smoothed_pred_label_id==4 or smoothed_pred_label_id==5)) or \
           (window_label_id==9 and (smoothed_pred_label_id==4 or smoothed_pred_label_id==6)) or \
           (window_label_id==10 and (smoothed_pred_label_id==6 or smoothed_pred_label_id==4)) or \
           (window_label_id==11 and (smoothed_pred_label_id==5 or smoothed_pred_label_id==6)) or \
           (window_label_id==12 and (smoothed_pred_label_id==6 or smoothed_pred_label_id==5)):
        
            correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

In [ ]:
folds_accu = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
folds_cao_3_accu = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
folds_krishnan_accu = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
folds_simple_voting_accu = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
folds_EWV_alpha_5_accu = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
folds_EWV_alpha_25_accu = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#folds_EWV_alpha_95_accu = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

for i in range(1,31):
    split_subject_id = i
    df_continuous = df_main.loc[df_main['subject_id'] == split_subject_id]
    df_continuous = df_continuous.reset_index(drop=True)

    df_OVA = df_main.loc[df_main['subject_id'] != split_subject_id]
    df_OVA = df_OVA.reset_index(drop=True)

    print ("fold" , i , "Training:")
    print ("len of df_main:" , str(len(df_main)))
    print ("len of df_continuous:" , str(len(df_continuous)))
    print ("len of df_OVA:" , str(len(df_OVA)))
    print ()
    
    models = train(df_OVA)
    
    print ("\nfold" , i , "Testing:")
    accu = test(df_continuous, models)
    folds_accu[i-1] = accu
    
    print ("\nCao et al Smoothing (length=3):")
    cao_3_accu = test_with_cao_smoothing_3 (df_continuous , models)
    folds_cao_3_accu[i-1] = cao_3_accu
    
    print ("\nKrishnan et al Smoothing (delta_t=3):")
    krishnan_accu = test_with_Krishnan_smoothing_delta3(df_continuous , models)
    folds_krishnan_accu[i-1] = krishnan_accu
    
    print ("\nSimple Voting Scheme Smoothing:")
    simple_voting_accu = test_with_simple_voting_smoothing(df_continuous , models)
    folds_simple_voting_accu[i-1] = simple_voting_accu
    
    print ("\nExponentially Weighted Voting (EWV) Smoothing (alpha=0.05):")
    EWV_alpha_5_accu = test_with_EWV_smoothing (df_continuous , models, 0.05)
    folds_EWV_alpha_5_accu[i-1] = EWV_alpha_5_accu

    print ("\nExponentially Weighted Voting (EWV) Smoothing (alpha=0.25):")
    EWV_alpha_25_accu = test_with_EWV_smoothing (df_continuous , models, 0.25)
    folds_EWV_alpha_25_accu[i-1] = EWV_alpha_25_accu

    #print ("\nExponentially Weighted Voting (EWV) Smoothing (alpha=0.95):")
    #EWV_alpha_95_accu = test_with_EWV_smoothing (df_continuous , models, 0.95)
    #folds_EWV_alpha_95_accu[i-1] = EWV_alpha_95_accu
    
    print()

In [ ]:
def get_avg_folds (lst):
    sum = 0
    for i in range(len(lst)):
        sum += lst[i]
    
    avg = sum / len(lst)
    return avg

In [ ]:
avg_accu = get_avg_folds(folds_accu)
print ("\nAverage Accuracy for all folds:" , avg_accu)

avg_cao_3_accu = get_avg_folds(folds_cao_3_accu)
print ("\nAverage Accuracy with Cao Smoothing (length=3) for all folds:" , avg_cao_3_accu)

avg_krishnan_accu = get_avg_folds(folds_krishnan_accu)
print ("\nAverage Accuracy with Krishnan Smoothing (delta_t=3) for all folds:" , avg_krishnan_accu)

avg_simple_voting_accu = get_avg_folds(folds_simple_voting_accu)
print ("\nAverage Accuracy with Simple Voting Scheme Smoothing for all folds:" , avg_simple_voting_accu)

avg_EWV_alpha_5_accu = get_avg_folds(folds_EWV_alpha_5_accu)
print ("\nAverage Accuracy with EWV Smoothing (alpha=0.05) for all folds:" , avg_EWV_alpha_5_accu)

avg_EWV_alpha_25_accu = get_avg_folds(folds_EWV_alpha_25_accu)
print ("\nAverage Accuracy with EWV Smoothing (alpha=0.25) for all folds:" , avg_EWV_alpha_25_accu)